#Descarregamento dos dados do ficheiro CVS_cleaned

In [ ]:
import pandas as pd
import numpy as np

# Ler o arquivo CSV
df = pd.read_csv('CVD_cleaned.csv')

FileNotFoundError: [WinError 2] O sistema não conseguiu localizar o ficheiro especificado

#Supervised Learning Algorithms

## Arvores de decisão

## Percepção de multi camada

## K-NN

#Unsupervised learning algorithms

## k-means

## DBScan

#Remoção de 10% e 20% dos dados

In [ ]:
def remover_dados(df,percentual_remocao):

  # Substituir 10% dos valores de cada linha por NaN
  for index, row in df.iterrows():
    # Selecionar colunas aleatórias na linha
    num_colunas = len(row)
    colunas_a_remover = np.random.choice(row.index, size=round(num_colunas * percentual_remocao), replace=False)
    # Substituir os valores escolhidos por NaN
    df.loc[index, colunas_a_remover] = np.nan
  return df

numero_registos = 10
df_copia = df.sample(n=numero_registos, random_state=42)
#print(df_copia.iloc[2])

df_delete_dez = remover_dados(df_copia,0.1)
df_delete_vinte = remover_dados(df_copia,0.2)

#Normalização dos dados

##Min-Max

##Z-score

#Discritização dos dados

## Equal-Width binning

## Equal-Height binning

#Redução dos dados

## Attribute Sampling

## Aggregation